In [ ]:
!pip install pretty_midi

In [1]:
import pretty_midi
import numpy as np
import copy

class midi_analysis:
    def __init__(self,filename):
        self.filename = filename
        self.tone_data(filename)

    def tone_data(self,filename):
        midi_data = pretty_midi.PrettyMIDI(filename) #start, end, pitch, velocity
        self.tempo = midi_data.estimate_tempo()
        self.melody = []
        self.word_count = []
        self.music_segment = []
        self.syllable_length = [[],[],[],[]]
        self.seg = []
        leng = 0
        k = -1 # counting note ID

        for i in midi_data.instruments[0].notes:
            k += 1 
            if k == 0: 
                temp = [midi_data.instruments[0].notes[0].pitch] # save first note 
                continue 
            if midi_data.instruments[0].notes[k].start == midi_data.instruments[0].notes[k-1].end: # check if 2nd note is following 1st note
                temp += [midi_data.instruments[0].notes[k].pitch]
            else:
                self.melody += [temp] # save entire sentence to self.melody
                temp = [midi_data.instruments[0].notes[k].pitch] # reset temp

        self.melody += [temp] # save last sentence

        for i in self.melody:
            self.word_count +=  [len(i)]
            
        for i in self.word_count:
            leng += i
            self.seg.append(leng)
        
        notes = midi_data.instruments[0].notes
        for i in range(0,len(notes)):
            if i <self.seg[0]:
                o = notes[i].end-notes[i].start
                self.syllable_length[0].append(o)
            elif i>=self.seg[0] and i<self.seg[1]:
                o = notes[i].end-notes[i].start
                self.syllable_length[1].append(o)       
            elif i>=self.seg[1] and i<self.seg[2]:
                o = notes[i].end-notes[i].start
                self.syllable_length[2].append(o)   
            else:
                o = notes[i].end-notes[i].start
                self.syllable_length[3].append(o)
        self.syllable_length2 = copy.deepcopy(self.syllable_length)
        self.syllable_length3 = copy.deepcopy(self.syllable_length)
        for i in range(len(self.syllable_length)):
            s = np.median(self.syllable_length[i])#找到中位数
            for x in range(len(self.syllable_length[i])):
                if self.syllable_length[i][x] <= s:
                    self.syllable_length2[i][x]=0#short note
                else:
                    self.syllable_length2[i][x]=1#long note
        for i in range(len(self.syllable_length)):
            s =0.4#absolute length
            for x in range(len(self.syllable_length[i])):
                if self.syllable_length[i][x] <= s:
                    self.syllable_length3[i][x]=0#short note
                else:
                    self.syllable_length3[i][x]=1#long note    
                    
        self.tone = []
        for i in range(len(self.melody)):
            sentence_tone = []
            for k in range(len(self.melody[i])):
                if k == 0:
                    if self.melody[i][k]>=self.melody[i][k+1]:
                        note_tone = [1,2,4]
                    else:
                        note_tone = [2,3]
                elif k == (len(self.melody[i])-1):
                    if self.melody[i][k-1]<self.melody[i][k]:
                        note_tone = [2,4]
                    elif self.melody[i][k-1]==self.melody[i][k]:
                        note_tone = [1,2]
                    else:
                        note_tone = [2,3]
                else:
                    note_tone = [2]
                    if self.melody[i][k-1]<=self.melody[i][k]:
                        note_tone += [1]
                    if self.melody[i][k-1]>self.melody[i][k] and self.melody[i][k]<self.melody[i][k+1]:
                        note_tone += [3]
                    if self.melody[i][k-1]<self.melody[i][k] and self.melody[i][k]>=self.melody[i][k+1]:
                        note_tone += [4]
                sentence_tone += [note_tone]
            self.tone += [sentence_tone]

if __name__ == "__main__":
    filename = "./9.mid"
    A=midi_analysis(filename)

FileNotFoundError: [Errno 2] No such file or directory: './9.mid'

In [3]:
def load_melody(midi_idx):
    root_path = "/home/yc429/Ch_lyrics_generation/12_new_melodies/"
    res = midi_analysis(root_path + str(midi_idx)+".mid")
    return res

load_melody(1).melody

[[59, 64, 71, 71, 67, 67, 69, 67, 66],
 [67, 66, 64, 67, 64, 61],
 [61, 64, 69, 67, 66, 69, 66, 69],
 [67, 66, 67, 67, 69, 67, 64]]

In [4]:
# Tone 1,2,3,4,5
# Relative pitch 4,3,1,5,0
# Rule 1. 轻声（5）不能出现在第一个位置，􀁘用力小，􀁙音强弱，􀁚音长短
# Rule 2. 三声不可能连续存在。当前一个位置是三声时，后一个就不可能是三声 
import numpy as np
import itertools
from numpy.linalg import norm
import copy

def calculate_deltas(numbers):#shifted
    mini = min(numbers)
    res = list()
    for num in numbers:
        res.append(num-mini)
    
    return res

# Example usage
input_numbers = [80, 78, 76, 76, 78]
delta_list = calculate_deltas(input_numbers)
print(delta_list)



def no_two_adjacent_zeros_tone3(lst):
    for i in range(len(lst) - 1):
        if lst[i] == 0 and lst[i+1] == 0:
            return False
        elif lst[i] == 1 and lst[i+1] == 1:
            return False
    return True

def all_tone_possibility(melody):
    n = len(melody)
    result = []
    # Generate all possible lists
    possible_lists = list(itertools.product([0,3,5], repeat=n-1))
    for first_element in [0,3,5]:
        for lst in possible_lists:
            result.append([first_element] + list(lst))
    result = list(filter(no_two_adjacent_zeros_tone3, result))

    # Filter out the lists that have the first element as 0
    result = list(filter(lambda x: x[0] != 0, result))


    return result

def forward_similarity(melody,lst):
    nan = [0] * len(melody)
    if melody != nan:
        melody = np.array(melody)
        next_lst = list()
        maxi = 0

        tone_pitch = [0,3,5]
        for i in tone_pitch:
            tmp = list()
            tmp = copy.deepcopy(lst)
            tmp = np.append(tmp, i)
            cos_sim = np.dot(melody,tmp)/ (norm(melody) * norm(tmp))

            if maxi < cos_sim:
                next_lst = tmp
                maxi = cos_sim
            else:
                pass
        return next_lst
    else:
        return np.array(melody)

forward_similarity([9, 4, 2, 4, 2,0,9],[5, 0, 3, 0, 0,3])

def best_1_scansion(input_num):#change number of candidates
    nan = [0] * len(input_num)
    if input_num != nan:
        best_tone_top = list()
        compare = 0
        deltas = calculate_deltas(input_num)
        for cd in all_tone_possibility(deltas[:5]):
            tmp = deltas[:5]
            cd = np.array(cd)
            norm_pitch = np.array(deltas)
            cos_sim = np.dot(cd, tmp) / (norm(cd) * norm(tmp))
    #         print(tmp,cd,cos_sim)
            if compare < cos_sim:
                best_tone_top = cd
                compare = cos_sim
    #             print("best",best_tone_top,cos_sim)
            else:
                pass

        for idx in range(5,len(deltas)):
            num = deltas[:idx +1]
            best_tone_top = forward_similarity(num,best_tone_top)

        return best_tone_top #relative pitch of tones
    else:
        print(input_num)
        return np.array(input_num)

def num2tone_label(best1):
    best1 = best1.tolist()
    tone_conversion = {0:"<l>",3:"<m>",5:"<h>"}
    return " ".join([tone_conversion[x] for x in best1])

[4, 2, 0, 0, 2]


In [5]:
for melody in A.melody:
    print(melody,num2tone_label(best_1_scansion(melody)))

[67, 64, 67, 71, 67, 64, 67, 67] <m> <l> <m> <h> <m> <l> <m> <m>
[67, 68, 67, 68, 65, 63, 67] <h> <h> <h> <h> <m> <l> <h>
[67, 64, 69, 67, 64, 67, 65, 67, 65, 71] <m> <l> <h> <m> <l> <m> <l> <m> <l> <h>
[65, 67, 67, 67, 69, 67, 64, 62, 60] <m> <h> <h> <h> <h> <h> <m> <l> <l>


# 3-height similarity

In [12]:
# mapping
symbol2num = {"<l>":"0","<m>":"3","<h>":"5"}

def forward_similarity(melody,lst):
    nan = [0] * len(melody)
    if melody != nan:
        melody = np.array(melody)
        next_lst = list()
        maxi = 0

        tone_pitch = [0,3,5]
        for i in tone_pitch:
            tmp = list()
            tmp = copy.deepcopy(lst)
            tmp = np.append(tmp, i)
            cos_sim = np.dot(melody,tmp)/ (norm(melody) * norm(tmp))

            if maxi < cos_sim:
                next_lst = tmp
                maxi = cos_sim
            else:
                pass
        return next_lst
    else:
        return np.array(melody)

forward_similarity([9, 4, 2, 4, 2,0,9],[5, 0, 3, 0, 0,3])

def best_1_scansion(input_num):#change number of candidates
    nan = [0] * len(input_num)
    if input_num != nan:
        best_tone_top = list()
        compare = 0
        deltas = calculate_deltas(input_num)
        for cd in all_tone_possibility(deltas[:5]):
            tmp = deltas[:5]
            cd = np.array(cd)
            norm_pitch = np.array(deltas)
            cos_sim = np.dot(cd, tmp) / (norm(cd) * norm(tmp))
    #         print(tmp,cd,cos_sim)
            if compare < cos_sim:
                best_tone_top = cd
                compare = cos_sim
    #             print("best",best_tone_top,cos_sim)
            else:
                pass

        for idx in range(5,len(deltas)):
            num = deltas[:idx +1]
            best_tone_top = forward_similarity(num,best_tone_top)

        return best_tone_top #relative pitch of tones
    else:
        print(input_num)
        return np.array(input_num)

In [13]:
# num to scansion symbol
num2_symbol = {3:"<m>",5:"<h>",0:"<l>"}

In [14]:
def contour_symbol(idx):
    contour_scansion = list()
    for line in load_melody(idx).melody:
        line = [x - min(line) for x in line]
        contour_scansion.append([num2_symbol[x] for x in best_1_scansion(line)])
    return contour_scansion

In [15]:
print([len(x) for x in contour_symbol(1)])
print([len(x) for x in rule_scansion_wb(1)])

[9, 6, 8, 7]
[9, 6, 8, 7]
